# Spatial demo with Jupyter Notebook and Exasol

### Prerequsities

Installing all python libraries required for this demo

In [ ]:
from IPython import get_ipython
if get_ipython() is None:
    from IPython.core.interactiveshell import InteractiveShell
    InteractiveShell.instance()
!pip install ipython-sql sqlalchemy-exasol folium pandas geojson requests jupyter_contrib_nbextensions geopy

Importing all required installed libraries to Jupyter Notebook

In [1]:
import folium
import pandas as pd
import os
import geojson
import warnings
import requests as r
import json
import geopy.geocoders
from geopy.geocoders import Nominatim
warnings.filterwarnings('ignore')

Load Jupyter magic functions

In [2]:
%reload_ext sql

Enter user credentials

In [5]:
EXA_USER = ""
EXA_PWD = ""
DSN = "exadb"

Connect to DSN

In [6]:
%sql exa+pyodbc://{EXA_USER}:{EXA_PWD}@{DSN}

'Connected: @None'

Set query cache off

In [ ]:
%sql alter session set query_cache='off';

### Datasets

#### Open schema NYC_UBER

In [ ]:
%sql open schema NYC_UBER

Overview of UBER_TAXI_DATA table

In [ ]:
%sql describe NYC_UBER.UBER_TAXI_DATA

Count of uber pickup records in UBER_TAXI_DATA table

In [ ]:
%sql select count(*) from NYC_UBER.UBER_TAXI_DATA

Date and time range of uber pickups 

In [7]:
%sql select min(DATETIME) as START_DATE,max(DATETIME) as END_DATE from NYC_UBER.UBER_TAXI_DATA

1 rows affected.


start_date,end_date
2014-04-01 00:00:00,2014-09-30 22:59:00


#### Open schema NYC_TAXI

In [ ]:
%sql open schema NYC_TAXI

Overview of TRIPS table

In [ ]:
%sql describe NYC_TAXI.TRIPS

Count of yellow taxi pickups records in TRIPS table

In [ ]:
%sql select count(*) from NYC_TAXI.TRIPS where CAB_TYPE_ID=1

Date and time range of New York City yellow taxi pickups 

In [8]:
%sql select min(PICKUP_DATETIME) as START_DATE,max(PICKUP_DATETIME) as END_DATE from NYC_TAXI.TRIPS

1 rows affected.


start_date,end_date
2009-01-01 00:00:00,2017-06-30 23:59:59


## Use Cases


### Uber pickups grouped by  New York City  boroughs

In the first use case, we use `DISJUNCT_NIGHBORHOODS` and `UBER_TAXI_DATA` tables from `NYC_UBER` schema to visualize uber pickups grouped by boroughs. The geometry column in `DISJUNCT_NEIGHBORHOODS` table contains polygons for boroughs while the geometry column in `UBER_TAXI_DATA` table from `NYC_UBER` schema contains pickup points  

Geometry column of type `Polygon` in `DISJUNCT_NEIGHBORHOODS`table 

In [ ]:
%sql select THE_GEOM from NYC_UBER.DISJUNCT_NEIGHBORHOODS limit 1

Geometry column of type `POINT` in `UBER_TAXI_DATA` table

In [ ]:
%sql select THE_GEOM from NYC_UBER.UBER_TAXI_DATA limit 1

Exasol automatically creates indices for equality join conditions, even when expressions are used for comparison.
Exasol 6.1 introduced indices on geospatial data types for joins using geospatial functions like ST_CONTAINS or ST_INTERSECTS. In this use case we use `ST_CONTAINS` function to join table `DISJUNCT_NEIGHBORHOODS` with `UBER_TAXI_DATA` on geometry columns grouped by New York City boroughs 

`%%time` is a cell magic function used here to calculate query execution time. `Wall time` gives the total of query runtime and cell rendering time (negligible)

In [9]:
%%time
%sql select borough_id, count(*) as pickups FROM NYC_UBER.DISJUNCT_NEIGHBORHOODS n INNER JOIN NYC_UBER.UBER_TAXI_DATA t ON ST_CONTAINS(n.THE_GEOM, t.THE_GEOM) group by borough_id order by pickups desc

5 rows affected.
Wall time: 56.7 ms


borough_id,pickups
Manhattan,3443402
Brooklyn,593648
Queens,342186
Bronx,31589
Staten Island,1034


Visualizing geospatial data of uber pickups grouped by  New York City  boroughs 

In [ ]:
#--- to be removed if direct links to http://data.beta.nyc works again---#

nyc_boroughs = 'geojsonfiles/nycboroughboundaries.geojson'

#nyc_boroughs = "http://data.beta.nyc//dataset/68c0332f-c3bb-4a78-a0c1-32af515892d6/resource/7c164faa-4458-4ff2-9ef0-09db00b509ef/download/42c737fd496f4d6683bba25fb0e86e1dnycboroughboundaries.geojson"

borough_pickups_sql = %sql select borough_id, count(*) as pickups FROM NYC_UBER.DISJUNCT_NEIGHBORHOODS n INNER JOIN NYC_UBER.UBER_TAXI_DATA t ON ST_CONTAINS(n.THE_GEOM, t.THE_GEOM) group by borough_id order by pickups desc
borough_pickups_df = borough_pickups_sql.DataFrame()

#base map
m1 = folium.Map([40.7586,-73.9706], zoom_start=10)

# Choropleth:
# geo_data: data of borough polygons
# Columns: 1st column is key (Borough) and 2nd column is value(total number of pickups)
# Key_on: Variable in the GeoJSON file to bind the data to
# bins = width between values
choropleth  = folium.Choropleth(geo_data=nyc_boroughs,name = 'choropleth', data = borough_pickups_df, columns = ['borough_id','pickups'],key_on='feature.properties.borough', fill_color='YlGnBu',bins=[1,100,300000,500000,600000,3500000],fill_opacity = 0.5,nan_fill_color='yellow' ,legend_name='Number of pickups', highlight=True).add_to(m1)

#hover over to view tooltip with borough name 
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['borough'])
)

# We can also export this interactive map to results/...html file
#m.save(os.path.join('results', 'GeoJSONWithoutTitles_2.html'))

# display map 
display(m1)

### Uber pickups grouped by New York City Neighborhoods

In the second use case, we use `DISJUNCT_NIGHBORHOODS` and `UBER_TAXI_DATA` tables from `NYC_UBER` schema to visualize uber pickups grouped by neighborhoods. The geometry column in `DISJUNCT_NEIGHBORHOODS` table contains polygons for neighborhoods while the geometry column in `UBER_TAXI_DATA` table from `NYC_UBER` schema contains pickup points  

Similar to the previous use case, we join neighborhood polygons with uber pickup points using `ST_CONTAINS` function to count total uber pickups grouped by New York City neighborhoods

In [ ]:
%%time
%sql select neighborhood,count(*) as pickups FROM NYC_UBER.DISJUNCT_NEIGHBORHOODS n INNER JOIN NYC_UBER.UBER_TAXI_DATA t ON ST_CONTAINS(n.THE_GEOM, t.THE_GEOM) group by neighborhood order by pickups desc limit 10

Visualizing geospatial data of uber pickups grouped by New York City neighborhoods

In [ ]:
#--- to be removed if direct links to data.beta.nyc works ---#

nyc_neighborhoods = 'geojsonfiles/nycneighborhoods.geojson'

#nyc_neighborhoods = "http://data.beta.nyc//dataset/0ff93d2d-90ba-457c-9f7e-39e47bf2ac5f/resource/35dd04fb-81b3-479b-a074-a27a37888ce7/download/d085e2f8d0b54d4590b1e7d1f35594c1pediacitiesnycneighborhoods.geojson"
neighborhood_pickups_sql = %sql select neighborhood,count(*) as pickups FROM NYC_UBER.DISJUNCT_NEIGHBORHOODS n INNER JOIN NYC_UBER.UBER_TAXI_DATA t ON ST_CONTAINS(n.THE_GEOM, t.THE_GEOM) group by neighborhood order by pickups desc 
neighborhood_pickups_df = neighborhood_pickups_sql.DataFrame()

#base map
m2 = folium.Map([40.7586,-73.9706], zoom_start=10)

# Choropleth:
# geo_data: data of borough polygons
# Columns: 1st column is key (Neighborhood) and 2nd column is value(total number of pickups). 
# Key_on: Variable in the GeoJSON file to bind the data to
# bins = width between values
# nan_fill_colors= yellow for neighborhoods with no pickup data
# For a detailed reference see https://python-visualization.github.io/folium/modules.html#Extra_Features

choropleth  = folium.Choropleth(geo_data=nyc_neighborhoods,name = 'choropleth', data = neighborhood_pickups_df, columns = ['neighborhood','pickups'],key_on='feature.properties.neighborhood', fill_color='YlOrRd',fill_opacity = 0.5, legend_name='Number of pickups',nan_fill_color='yellow',nan_fill_opacity=0.4,bins=[1,560,114694,181979,349255,666970],highlight=True).add_to(m2)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['neighborhood'])
)


# We can also export this interactive map to results/...html file
#m.save(os.path.join('results', 'GeoJSONWithoutTitles_2.html'))

# display map with choropleth
display(m2)

### New York City Streets with highest Uber pickups

In this use case, we use NYC street data and NYC Uber pickup data to visualize top streets according to number of pickups. Lets have a look at `STREETS` table from `NYC_UBER` schema. `PHYSICALID`, `THE_GEOM` and `ST_NAME` are columns used for this demo

In [ ]:
%sql describe NYC_UBER.STREETS

Before querying for New York City streets with highest Uber pickups, we create a view from `STREETS` table and transform the geometry column from a spherical coordinate system(SRID:4326) to a Mercator cordinate system(SRID:3857) using `ST_TRANSFORM` function. The transformation from 4326 WGS84 (spherical coordinates) to 3857 (Google) Mercator has the advantage that for Mercator, distance is measured in meters (in contrast to 4326 where distance is measured in degrees).  Mercator is used by most of the map services including OpenStreetMap (used in this demo). After transformation a buffer of 50 meters is added around the street geometry column using `ST_BUFFER` function to account for positioning inaccuracy. A snapshot of `STREETS_TRANSFORMED` view:

``` mysql
CREATE OR REPLACE VIEW "NYC_UBER"."STREETS_TRANSFORMED" as select
...
ST_BUFFER(ST_TRANSFORM(THE_GEOM, 3857),50) as THE_GEOM,
...
from NYC_UBER.STREETS;
```

Similary the geometry column from `UBER_TAXI_DATA` is transformed to Mercator using `ST_TRANSFORM` function. A snapshot of the transformed `UBER_TAXI_DATA_TRANSFORMED` view: 

``` mysql
CREATE OR REPLACE VIEW "NYC_UBER"."UBER_TAXI_DATA_TRANSFORMED"
...
as select DATETIME,LAT,LON, BASE, ST_TRANSFORM(the_geom,3857) as the_geom 
...
from UBER_TAXI_DATA;
```


Select the number of streets with highest pickups to view on map

In [ ]:
NumberOfStreets = 5

EXASOL query joins the views based on the geometry columns using `ST_CONTAINS`

In [ ]:
%%time
%sql select s.full_stree as street_name,count(*) as pickups from (select * from "NYC_UBER"."STREETS_TRANSFORMED" order by false) s INNER JOIN (select * from "NYC_UBER"."UBER_TAXI_DATA_TRANSFORMED" order by false) t ON ST_CONTAINS(s.the_geom,t.the_geom) group by s.full_stree order by pickups desc limit $NumberOfStreets

Visualizing geospatial data of uber pickups grouped by New York City streets

In [ ]:
#---- to be removed it API call gets fixed: atm returns only 1000 rows -----#

#data = open('C:/Users/smha/GeoSpatialViz/geojsonfiles/nyc_street_data.geojson','r')
#jsondata = json.loads(data)
path = 'geojsonfiles/nyc_street_data.geojson'
with open(path) as f:
    data = geojson.load(f)
features = data['features'][0]

#---------------------------------------------------------------------------#
#---------------  Instead use API call to data endpoint --------------------#

# url = 'https://data.cityofnewyork.us/resource/gr6w-nsbv.json'

# # get json street data by from NYC city data API
# req = r.get('https://data.cityofnewyork.us/resource/gr6w-nsbv.json')
# jsondata = json.loads(req.text)


# # convert json to geojson for folium choropleth
# GeoJSON = []
# for i in range(0,len(jsondata)-1):
#    GeoJSON.append(
#    {
#          "type": "Feature", 
#          "properties":
#              {
#              "physicalid": jsondata[i]["physicalid"],
#              "full_stree": jsondata[i]["full_stree"],
#              },
#           "geometry": jsondata[i]['the_geom'],
#    } )
    
# GeoJSON[0]
# data= {"type": "FeatureCollection","features": GeoJSON }

#top street SQL inline magic + EXASOL query
top5streets_sql = %sql select s.full_stree,count(*) as pickups from (select * from "NYC_UBER"."STREETS_TRANSFORMED" order by false) s INNER JOIN (select * from "NYC_UBER"."UBER_TAXI_DATA_TRANSFORMED" order by false) t ON ST_CONTAINS(s.the_geom,t.the_geom) group by s.full_stree order by pickups desc limit $NumberOfStreets
top5streets_df = top5streets_sql.DataFrame()

# taking top x 'physicalid's as a type string
top5streets_df['full_stree'] = top5streets_df.full_stree.astype(str)

# save column 'physicalid' from top street dataframe for the next steps
dfList = list(top5streets_df['full_stree'])

# match full street column names with street names from json and save correspoding cordinates to a list  
l = list()    
for i in range(0,len(data['features'])-1):  
  if data['features'][i]['properties']['full_stree'] in dfList:  
     l.append(data['features'][i])
     

# create a new dataframe with only the selected street geometry points
data['features'] = l
streetdata = json.dumps(data)
#base map

m3 = folium.Map([40.7586,-73.9706], zoom_start=12)

# Choropleth
# geo_data: data of borough polygons
# Key_on: Variable in the GeoJSON file to bind the data to
# bins = width bins between values
# For a detailed reference see https://python-visualization.github.io/folium/modules.html#Extra_Features

choropleth = folium.Choropleth(geo_data=streetdata,name = 'choropleth',key_on='feature.properties.full_stree', fill_color='YlGnBu',line_color = 'blue', line_weight= 5 , highlight=True).add_to(m3)

choropleth.geojson.add_child(
   folium.features.GeoJsonTooltip(['full_stree'])
)

# We can also export this interactive map to results/...html file
# m3.save(os.path.join('results', 'GeoJSONWithoutTitles_5.html'))

# display map with choropleth
display(m3)

### Comparison of Taxi and Uber pickups within a certain radius of a location in New York City 

The following use case compares the number of Uber and Yellow Taxi pickups. For this example we have selected `Museum of the City of New York` in Manhattan as a pickup point. We have used geocoding to find the latitude and longitude values of a given location. To visualize geospatial data on map for a different location, change the value of `pos` variable. The value of month can be adjusted to visualize different results. Radius defines the radius around the given lat/long point. For speed purposes its recommended to keep radius value small. 

Assigning query parameters 

In [ ]:
pos = "Museum of the City of New York"
geolocator = Nominatim()
geo = geolocator.geocode(pos, timeout=None) 
location_latitude = geo.latitude
location_longitude = geo.longitude
month = 6
radius = 100
geo_point = f"\'POINT({location_longitude} {location_latitude})\'"

`ST_SETSRID` geospatial function is used to set the SRID(Spatial reference system identifier) of the given `geo_point`

``` mysql
st_setsrid($geo_point,4326)
```

After setting the SRID, the given `geopoint` is transformed to Mercator using `ST_TRANSFORM` function. To count the number of pickups within the radius of the given `geopoint` we use `ST_DISTANCE` function. `ST_DISTANCE` function calculates the distance between two geospatial points. 

``` mysql
st_distance(st_transform(st_setsrid($geo_point,4326),3857),the_geom) < 100
```

Querying EXASOL to list pickup points for New York City yellow taxi given the above parameters

In [ ]:
%%time
%sql select pickup_latitude, pickup_longitude from nyc_taxi.trips where id in (select id from nyc_uber.nyc_taxi_with_point where st_distance(st_transform(st_setsrid($geo_point,4326),3857),the_geom) < $radius and year(pickup_date)=2014 and month(pickup_date)=$month) and CAB_TYPE_ID=1 

Querying EXASOL to list pickup points for Uber given the above parameters

In [ ]:
%%time
%sql select lat,lon from nyc_uber.uber_taxi_data_transformed where st_distance(st_transform(st_setsrid($geo_point,4326),3857),the_geom) < $radius and year(datetime)=2014 and month(datetime)=$month

Visualizing geospatial data comparing Taxi and Uber pickups within a certain radius of a location in New York City

In [ ]:
nyc_jfk = %sql select pickup_latitude, pickup_longitude from nyc_taxi.trips where id in (select id from nyc_uber.nyc_taxi_with_point where st_distance(st_transform(st_setsrid($geo_point,4326),3857),the_geom) < $radius and year(pickup_date)=2014 and month(pickup_date)=$month)
taxi_df = nyc_jfk.DataFrame()

uber_JFK = %sql select * from nyc_uber.uber_taxi_data_transformed where st_distance(st_transform(st_setsrid($geo_point,4326),3857),the_geom) < $radius  and year(datetime)=2014 and month(datetime)=$month
uber_df = uber_JFK.DataFrame()

#base map
emp_m = folium.Map([location_latitude,location_longitude], zoom_start=20)

# Add markers to pickup points on the map object 
for i in range(0,taxi_df.shape[0]-1):
    folium.Marker([taxi_df.iloc[i]['pickup_latitude'], taxi_df.iloc[i]['pickup_longitude']],icon=folium.Icon(color='orange', icon='taxi')).add_to(emp_m)
for i in range(0,uber_df.shape[0]-1):
    folium.Marker([uber_df.iloc[i]['lat'], uber_df.iloc[i]['lon']]).add_to(emp_m)  

display(emp_m)